In [1]:
import synergy_scraper as ss

In [2]:
bs = ss.CumulativeBoxScore("boxscore.html")
bs.df()

,Player,GP,Poss,Pts,PPP,Ast,TO,TO%,Ast/TO,FG Att,...,Chg Tkn,Chg Com,PF TKN,PF COM,Tot Reb %,Off Reb %,Def Reb %,Ast %,Stl %,Blk %
0,#43 Pavlos Stavrinides,7,111,104,0.937,12,32,28.8%,0.38,63,...,-,-,28,25,42.342342,10.810811,31.531532,10.810811,3.603604,4.504505
1,#13 Chase Pagon,7,137,102,0.745,22,24,17.5%,0.92,93,...,-,-,31,19,41.605839,13.868613,27.737226,16.058394,4.379562,2.189781
2,#22 Kyle Mcgraw,7,96,73,0.760,10,17,17.7%,0.59,74,...,-,-,8,16,32.291667,9.375000,22.916667,10.416667,8.333333,2.083333
3,#4 Noah Hicks,7,92,59,0.641,16,22,23.9%,0.73,66,...,-,-,10,8,31.521739,5.434783,26.086957,17.391304,1.086957,0.000000
4,#23 Riley Wood,7,43,28,0.651,9,12,27.9%,0.75,20,...,-,-,12,12,113.953488,44.186047,69.767442,20.930233,13.953488,2.325581
5,#34 Evan Portnoi,7,57,28,0.491,7,12,21.1%,0.58,41,...,-,-,9,5,17.543860,0.000000,17.543860,12.280702,1.754386,5.263158
6,#21 Ben Juarez,6,32,28,0.875,4,3,9.4%,1.33,28,...,-,-,3,6,50.000000,9.375000,40.625000,12.500000,3.125000,0.000000
7,#0 Sidd Shendrikar,4,10,4,0.400,3,6,60.0%,0.50,4,...,-,-,0,2,0.000000,0.000000,0.000000,30.000000,0.000000,0.000000
8,#11 Jack Pierson,3,10,3,0.300,1,2,20.0%,0.50,8,...,-,-,0,0,30.000000,10.000000,20.000000,10.000000,0.000000,0.000000
9,#24 James Downs,3,4,0,0.000,2,0,0.0%,-,3,...,-,-,1,2,50.000000,50.000000,0.000000,50.000000,0.000000,0.000000
